# Testing in-memory data loads and exports with Pandas

In this notebook, the loading (in-memory) and exporting capabilities of Pandas are tested. The following steps are processes: <br>

- Loading data from multiple csv files into one dataframe (around 20 files of different sizes)
- Loading data from one big csv file (3.2 million rows)
- Exporting the data from dataframe and table into a parquet file.

Prerequisite to run this script: 
- The preceding notebook "data_setup/00_Get_Data.ipynb" must be executed before which stores the csv file under the referred directories.
- **Increase buffer size** when running locally  <code> jupyter notebook --NotebookApp.max_buffer_size = your_value*</code>
 
 *your_value = desired buffer size in bytes

In [1]:
import os
import glob
import time

import pandas as pd

In [2]:
# Initial variable for the directory
path = os.getcwd()

# Test 1: Loading from Multiple Files

In [3]:
# Path to the GWR csv files on canton level
os.chdir(path)
os.chdir("..\datasets\GWR")

### Loading with Pandas (in-memory)

In [4]:
ct = time.time()
cantons_df_pandas = pd.concat([pd.read_csv(f, on_bad_lines='skip', sep='\t') for f in glob.glob('*.csv')])
print(f"Pandas load time: {(time.time() - ct)}")
print(f"Number of dataframe rows: {cantons_df_pandas.shape[0]}")

Pandas load time: 4.898589134216309
Number of dataframe rows: 2573168


In [5]:
# Checking the structure
cantons_df_pandas.head(1)

,EGID,EDID,EGAID,DEINR,ESID,STRNAME,STRNAMK,STRINDX,STRSP,STROFFIZIEL,DPLZ4,DPLZZ,DPLZNAME,DKODE,DKODN,DOFFADR,DEXPDAT
0,520001,0,100428604,43,10109554.0,Schiffländestrasse,Schiffländestr.,Sch,9901.0,1.0,5000,0,Aarau,2645359.617,1249365.315,1.0,2023-06-24


# Test 2: Loading from Single File

In [3]:
# Path to the single GWR csv file (whole Switzerland)
os.chdir(path)
os.chdir("..\datasets\GWR\GWR_Total")

### Loading with Pandas (in-memory)

In [9]:
ct = time.time()
switzerland_df_pandas = pd.read_csv('eingang_entree_entrata_total.csv', on_bad_lines='skip', sep='\t') 
print(f"Pandas load time: {(time.time() - ct)}")
print(f"Number of dataframe rows: {switzerland_df_pandas.shape[0]}")

Pandas load time: 4.690452814102173
Number of dataframe rows: 3262598


## Initial Results: 
In the **first scenario**, DuckDB was up to **2.5sec faster** than Pandas. Or when considered the whole loading time, around **50% faster**. However, this advantage play only out as long as the referenced object fits into memory. For the **second scenario**, DuckDB also performed better. However, DuckDB was not able to read the data in one transaction first. This works only with an increased memory buffer for the notebook kernel.
<br> <br>
Also worth mentioning: There is less code required in DuckDB to achieve the same results. For instance, no separator sign needs to be determined in DuckDB when reading the csv file. Conversely in Pandas, there will be an error when using a CSV file where the data is not separated with a comma.

# Export dataframe to parquet?

In [ ]:
ct = time.time()
df.to_parquet(path=None, engine='auto', compression='snappy',
              index=None, partition_cols=None, storage_options=None, **kwargs)
print(f"Pandas load time: {(time.time() - ct)}")